Features:

- Days

- Rainfall indicates the quantity of rain falling (mm)

- Temperature indicates the temperature (°C)

- Volume indicates the volume of water taken from the drinking water treatment plant (m**3)

- Hydrometry indicates the groundwater level (m)

Target:

- Depth to Groundwater indicates the groundwater level (m from the ground floor)



---

\\

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.preprocessing import RobustScaler

Days (0-365)

In [2]:
days = []
for i in range(1,366,1):
  days.append(i)
days

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

Depth Ground Water (30-32m Max limit of changes in depth)

In [3]:
from random import uniform

x = 1
depth = []
while x!=366:
    depth.append(round(uniform(30, 32), 2)) # water level randomised between 30 & 40 m
    x+=1

Rain (India average: 400 - 500 mm) 

In [4]:
import random

x = 1
rain = []
while x!=366:
    rain.append(round(uniform(400, 500), 2)) 
    x+=1

for x in range(200): # Putting Zeroes in rain as everyday no rainfall
    i = random.randint(0,365)
    rain[i] = 0

In [5]:
# Number of zeroes in rain

x = 0
c = 0
while x < 365:
  if rain[x] == 0:
      c = c+1
  x = x+1
c

152

Temperature (15 - 25 degree celsius)

In [6]:
x = 1
temp = []
while x!=366:
    temp.append(round(uniform(16, 25), 2)) 
    x+=1

Hydrometry / Groundwater Level (15-20) ***(Target)***

In [7]:
x = 1
level = []
while x!=366:
    level.append(round(uniform(15, 20), 2)) 
    x+=1

Joining Columns

In [8]:
arr = [days, depth, rain, temp, level] # put all columns in array and convert to dataframe.
arr = pd.DataFrame(arr) # convert
arr = arr.T # Transposing/Rotating by 90 degree
arr.columns = ['days', 'depth', 'rain', 'temp', 'level']
arr = arr.drop(['days'], axis = 1) # Days not reqd. Index is sufficient
arr

,depth,rain,temp,level
0,30.23,433.13,21.04,17.68
1,31.83,416.98,24.86,17.12
2,30.29,494.76,19.38,15.64
3,31.24,472.95,21.06,17.89
4,31.28,0.00,24.33,16.40
...,...,...,...,...
360,30.46,407.80,18.04,17.73
361,31.12,0.00,16.83,16.89
362,31.51,497.96,23.79,19.26
363,30.79,0.00,24.22,16.56


Features Column

In [9]:
X = arr.iloc[:,0:-1] # [row1:row2, column1:column2]

Target Column

In [10]:
Y = arr['level']
# arr.to_csv('data.csv')

# LSTM 1

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle=False, random_state=42)

In [27]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
 
# split a univariate sequence
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
#raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps = 3

# split into samples
#X, y = split_sequence(raw_seq, n_steps)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
#X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X_train, y_train, epochs=200, verbose=1)

Epoch 1/200
9/9 [==============================] - 2s 7ms/step - loss: 352.2766
Epoch 2/200
9/9 [==============================] - 0s 8ms/step - loss: 275.7045
Epoch 3/200
9/9 [==============================] - 0s 7ms/step - loss: 146.8061
Epoch 4/200
9/9 [==============================] - 0s 6ms/step - loss: 104.0542
Epoch 5/200
9/9 [==============================] - 0s 6ms/step - loss: 64.0290
Epoch 6/200
9/9 [==============================] - 0s 8ms/step - loss: 10.7077
Epoch 7/200
9/9 [==============================] - 0s 6ms/step - loss: 7.4171
Epoch 8/200
9/9 [==============================] - 0s 6ms/step - loss: 4.1187
Epoch 9/200
9/9 [==============================] - 0s 6ms/step - loss: 2.6249
Epoch 10/200
9/9 [==============================] - 0s 5ms/step - loss: 2.5540
Epoch 11/200
9/9 [==============================] - 0s 6ms/step - loss: 2.4044
Epoch 12/200
9/9 [==============================] - 0s 6ms/step - loss: 2.4206
Epoch 13/200
9/9 [==============================] -

In [28]:
# demonstrate prediction
n = 10
x_input = array(X_test[0:n])
x_input = x_input.reshape((n, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[18.078762]
 [18.378275]
 [18.169128]
 [18.384935]
 [18.191893]
 [17.80762 ]
 [18.38994 ]
 [18.06505 ]
 [17.884626]
 [18.016039]]


In [29]:
y_actual = y_test[0:n]
print(y_actual)

273    18.89
274    16.82
275    15.67
276    19.68
277    15.88
278    18.10
279    19.84
280    18.37
281    15.88
282    18.14
Name: level, dtype: float64


The input to every LSTM layer must be three-dimensional.

The three dimensions of this input are:

- **Samples:** One sequence is one sample. A batch is comprised of one or more samples.

- **Time Steps:** One time step is one point of observation in the sample.

- **Features:** One feature is one observation at a time step.


[samples, timesteps, features]